In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import matplotlib.pyplot as plt 
from string import punctuation

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [4]:
playlist = pd.read_csv('pre_total_playlist.csv')

## 불용어

In [6]:
len(stop_w)

120

## 신조어

In [9]:
len(add_words)

236

In [10]:
from ckonlpy.tag import Twitter
twitter = Twitter()

In [11]:
# 리스트에 담긴 단어만큼 사전에 추가
for i in range(len(add_words)):
    twitter.add_dictionary(add_words[i], 'Noun')

## 토큰화

In [13]:
word_list = playlist['Lyric'].apply(lambda x: [word for word in twitter.nouns(x) if word not in stop_w])

In [14]:
word_list

0        [아무, 리, 바보, 울, 곁, 상처, 기, 다리, 란, 보고, 보고, 만큼, 울,...
1        [그대, 란, 사람, 허락, 맘, 그대, 때문, 란, 눈길, 줄, 만큼, 다만, 가...
2        [어쩌면, 오늘이, 마지막, 지도, 사랑, 불안, 마음, 니, 눈, 마주, 치, 사...
3        [술, 잔, 생각, 밤, 시절, 모두, 한숨, 그대, 얼굴, 혹시, 울, 지나, 먼...
4        [손짓, 목소리, 마음, 지도, 맘대로, 나라, 사랑, 행복, 맘, 프라, 가기, ...
                               ...                        
15337    [언제, 익숙, 매일, 반복, 하루, 연속, 모습, 어가, 거울, 얼굴, 위로, 주...
15338    [옛날, 옛적, 서울, 이란, 곳, 구월, 손톱, 구월, 구월, 기, 금, 간, 발...
15339    [일로, 전화, 술, 시간, 사실, 취하, 거, 우리, 안, 만해, 남아, 전화, ...
15340    [원래, 별, 뜻, 다정, 생각, 향, 길이, 중이, 언제, 순간, 가득, 차, 오...
15341    [아침, 진, 꽤, 거, 늦, 어도, 래서, 남, 맘, 아침, 선택, 사항, 사이,...
Name: Lyric, Length: 15342, dtype: object

## 벡터화

* min_df : 단어장에 포함되기 위한 최소 빈도
* max_df : 단어장에 포함되기 위한 최대 빈도
* ngram_range : 튜플 (min_n, max_n) n-그램 범위

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 설정해준 카테고리의 데이터들만 추출
# CountVectorizer로 텍스트 데이터들 단어 빈도수에 기반해 벡터화시키기(fit_transform)
count_vect = CountVectorizer(max_df=0.95, max_features=1000,
                            min_df=2, stop_words=stop_w,
                            ngram_range=(1,5)) 
ftr_vect = count_vect.fit_transform(word_list.astype(str))


## 토픽 모델링

In [16]:
# LDA클래스를 이용해서 피처 벡터화시킨 것을 토픽모델링 시키기
# n_components(토픽개수) 3로 설정
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(ftr_vect)
# components_속성은 3개의 토픽별(row)로 1000개의 feature(단어)들의 분포수치(column)를 보여줌
print(lda.components_.shape)
print(lda.components_)

(3, 1000)
[[ 21.19249197  50.24118538 318.24898069 ...  14.8570299  152.10444304
   14.27736526]
 [ 23.53864738  43.52012048  31.00857055 ...  69.43079688 161.9596659
   58.25934253]
 [115.26886065 328.23869414 414.74244876 ... 125.71217322  51.93589106
   83.46329221]]


In [29]:
# transform까지 수행하면, 문서별(row)로 토픽들(column)의 분포를 알려줌
doc_topics = lda.transform(ftr_vect)
print(doc_topics.shape)
print(doc_topics[:2]) # 두 곡에 대한 확률 정보

(15342, 3)
[[0.00838982 0.98303465 0.00857553]
 [0.51127507 0.4798901  0.00883483]]


## 토픽 별 주요 단어

In [22]:
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx+1)
        
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌

        topic_word_idx = topic.argsort()[::-1] # argsort()는 디폴트가 오름차순, [::-1]로 내림차순 변경
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들)볼 수 있음
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일!
        # '문자열'.join 함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음.
        feature_concat = '+'.join([str(feature_names[i])+'*'+str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)        
        
feature_names = count_vect.get_feature_names()
display_topic_words(lda, feature_names, 15)


Topic # 1
그대*14800.4+사랑*6545.2+마음*3584.4+우리*3564.0+어요*3175.4+아름*2593.8+바람*2480.5+기억*2142.7+하루*2079.1+오늘*2077.1+시간*1843.3+순간*1821.8+다가*1760.2+가득*1659.1+하늘*1636.2

Topic # 2
사랑*17340.2+우리*5628.8+사람*5448.9+마음*3698.7+눈물*3610.6+시간*3397.9+기억*3083.0+아직*2951.6+행복*2906.3+생각*2487.2+이별*2129.1+아무*2087.9+세상*1994.9+가슴*1833.5+모습*1804.8

Topic # 3
오늘*5906.3+지금*5193.9+우리*5000.3+생각*4436.1+아무*4232.0+어디*3314.5+시간*3038.8+매일*2977.5+하루*2589.6+다가*2236.5+조금*2161.9+기분*2161.4+마음*2077.9+모두*2060.0+내일*2005.4


## 가장 높은 확률의 토픽만 추출

In [25]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌

doc_topic = lda.transform(ftr_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,1,0.983035
1,1,0,0.511275
2,2,1,0.879037
3,3,0,0.459547
4,4,1,0.985411


## 노래 별 토픽 확률

In [30]:
# 주어진 내장 텍스트데이터의 문서이름에는 카테고리가 labeling되어있음. 
# 따라서, 카테고리가 무엇인지 아는 상태이니까 어떤 문서들이 어떤 토픽들이 높은지 확인해보자.
# 그리고 그 토픽들이 각각 무엇을 내용으로 하는지 추측해보자.
# 주어진 데이터셋의 filename속성을 이용해서 카테고리값들 가져오기
def get_filename_list(playlist):
    filename_lst = []
    for file in playlist.song_name: 
        filename_temp = file.split('/')[-2:]
        filename = '.'.join(filename_temp)
        filename_lst.append(filename)
    return filename_lst
 
filename_lst = get_filename_list(playlist)
# Dataframe형태로 만들어보기
topic_names = ['Topic #'+ str(i) for i in range(1,4)]
topic_df = pd.DataFrame(data=doc_topics, columns=topic_names,
                       index=filename_lst)

topic_df.head()

,Topic #1,Topic #2,Topic #3
보고 싶다,0.008390,0.983035,0.008576
사랑합니다...,0.511275,0.479890,0.008835
어쩌면...,0.116552,0.879037,0.004410
소주 한 잔,0.459547,0.455085,0.085369
With Me,0.006774,0.985411,0.007815


## Topic # 1
**그대*14800.4+사랑*6545.2+마음*3584.4+우리*3564.0+어요*3175.4**+아름*2593.8+바람*2480.5+기억*2142.7+하루*2079.1+오늘*2077.1+시간*1843.3+순간*1821.8+다가*1760.2+가득*1659.1+하늘*1636.2

## Topic # 2
**사랑*17340.2+우리*5628.8+사람*5448.9+마음*3698.7+눈물*3610.6**+시간*3397.9+기억*3083.0+아직*2951.6+행복*2906.3+생각*2487.2+이별*2129.1+아무*2087.9+세상*1994.9+가슴*1833.5+모습*1804.8

## Topic # 3
**오늘*5906.3+지금*5193.9+우리*5000.3+생각*4436.1+아무*4232.0**+어디*3314.5+시간*3038.8+매일*2977.5+하루*2589.6+다가*2236.5+조금*2161.9+기분*2161.4+마음*2077.9+모두*2060.0+내일*2005.4

## 가장 높은 확률의 토픽

In [31]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌

doc_topic = lda.transform(ftr_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,1,0.983035
1,1,0,0.511275
2,2,1,0.879037
3,3,0,0.459547
4,4,1,0.985411


In [35]:
playlist.head(1) # 기본 곡 정보

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,418253,보고 싶다,김범수,3rd. 보고 싶다,342238,아무리 기다려도 난 못가 바보처럼 울고 있는 너의 곁에 상처만 주는 나를 왜 ...,https://image.bugsm.co.kr/album/images/200/262...,"['국내', '가요', '발라드한', '발라드', '국내 발라드', '가을', '쓸...","['아무리', '기다려도', '난', '못가', '바보처럼', '울고', '있는',..."


In [33]:
doc_topic_df = doc_topic_df.join(playlist,lsuffix='index') # 병합
doc_topic_df.drop('Doc_Num', axis=1,inplace=True)
doc_topic_df.head()  

,Topic,Percentage,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,1,0.983035,418253,보고 싶다,김범수,3rd. 보고 싶다,342238,아무리 기다려도 난 못가 바보처럼 울고 있는 너의 곁에 상처만 주는 나를 왜 ...,https://image.bugsm.co.kr/album/images/200/262...,"['국내', '가요', '발라드한', '발라드', '국내 발라드', '가을', '쓸...","['아무리', '기다려도', '난', '못가', '바보처럼', '울고', '있는',..."
1,0,0.511275,445413,사랑합니다...,팀,Tim 영민,332810,나빠요 참 그대란 사람 허락도 없이 왜 내 맘 가져요 그대 때문에 난 힘겹게 살고만...,https://image.bugsm.co.kr/album/images/200/307...,"['국내', '발라드', '발라드한', '100시리즈', '가을', '가요', '감...","['나빠요', '참', '그대란', '사람', '허락도', '없이', '왜', '내..."
2,1,0.879037,489056,어쩌면...,버즈,Morning Of Buzz,295142,어쩌면 오늘이 마지막이 될지도 몰라 나의 사랑이 떠날지 몰라 불안한 나의 마음 니가...,https://image.bugsm.co.kr/album/images/200/328...,"['국내', '락발라드', '발라드한', '2000년대', '세부장르', '100시...","['어쩌면', '오늘이', '마지막이', '될지도', '몰라', '나의', '사랑이..."
3,0,0.459547,460396,소주 한 잔,임창정,Bye,277349,술이 한 잔 생각나는 밤 같이 있는 것 같아요 그 좋았던 시절들 이젠 모두 한숨만...,https://image.bugsm.co.kr/album/images/200/314...,"['국내', '발라드한', '발라드', '국내 발라드', '100시리즈', '가요'...","['술이', '한', '잔', '생각나는', '밤', '같이', '있는', '것',..."
4,1,0.985411,482653,With Me,휘성(Realslow),It's Real,268787,네 손짓 하나 보는 게 난 좋은데 네 목소리를 듣는 것도 좋은데 왜 넌 내 ...,https://image.bugsm.co.kr/album/images/200/325...,"['가요', '섹시한', '남성보컬']","['네', '손짓', '하나', '보는', '게', '난', '좋은데', '네', ..."


In [34]:
# 토픽 별 곡 개수
doc_topic_df.groupby('Topic')[['song_id']].count()

,song_id
Topic,
0,3879
1,4586
2,6877


In [36]:
doc_topic_df.to_csv('total_topic_modeling.csv',index=False)